In [1049]:
import requests
from bs4 import BeautifulSoup
import json
from time import sleep
from datetime import datetime
import time
from random import randint
import urllib.request

In [1050]:
with requests.Session() as se:
    se.headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
        "Accept-Encoding": "gzip, deflate",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en"
        }

In [1051]:
kickstarter_projects = []

for i in range(0, 10):
    url = 'https://www.kickstarter.com/discover/advanced?state=successful&sort=end_date&seed=2744001&page='+str(i)
    response=se.get(url)
    print(response.status_code)
    soup=BeautifulSoup(response.content, "html.parser")
    data = [
    (json.loads(i["data-project"]), i["data-ref"])
        for i in soup.find_all("div")
        if i.get("data-project")
    ]
    kickstarter_projects.extend(data)
    sleep(randint(1,2))

200
200
200
200
200
200
200
200
200
200


In [1052]:
# cleaning
kickstarter_clean = [i[0] for i in kickstarter_projects]

In [1053]:
# getting only projects within last week

day = 24*60*60*7
kickstarter_recent = []

for i in range(len(kickstarter_clean)):
    if kickstarter_clean[i]['deadline'] > time.time() - day:
        kickstarter_recent.append(kickstarter_clean[i])

In [1054]:
len(kickstarter_recent), len(kickstarter_clean)

(120, 120)

In [1055]:
# getting the identified status of the company

In [1056]:
for i in range(len(kickstarter_recent)):

    try:
        url = 'https://www.kickstarter.com/profile/' + str(kickstarter_recent[i]['creator']['id']) + '/about'
        response=se.get(url)
        print(response.status_code)
        soup=BeautifulSoup(response.content, "html.parser")

        if soup.find_all('ul', {'class':'menu-submenu mb6'}):
            a_tags = soup.find_all('ul', {'class':'menu-submenu mb6'})[0].find_all('a')
        
        hrefs = []
        for a in a_tags:
            hrefs.append(a.get('href'))

        if len(hrefs) != 0:
            kickstarter_recent[i]['identified'] = True
            kickstarter_recent[i]['website'] = hrefs[0]
        else:
            kickstarter_recent[i]['identified'] = False
        
        sleep(randint(1,2))
        
    except:
        pass

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [1057]:
# filtering out unidentified companies
kickstarter_id = [i for i in kickstarter_recent if i['identified'] == True]

In [1058]:
len(kickstarter_recent), len(kickstarter_id)

(120, 120)

In [1059]:
# using builtwith to filter for domains using shopify

domain_info = []

for i in range(len(kickstarter_id)):
    with urllib.request.urlopen("https://api.builtwith.com/free1/api.[json]?KEY=&LOOKUP==" + str(kickstarter_id[i]['website'])) as url:
        data = json.loads(url.read().decode())
        domain_info.append(data)
        sleep(1)

In [1060]:
# filtering out non relevant domains

relevent_domains = []

for d in range(len(domain_info)):

    if 'first' in domain_info[d]:
        relevent_domains.append(domain_info[d])

In [1061]:
# checking for shopify in domains

shopify_websites = []

for d in range(len(relevent_domains)):
    for i in relevent_domains[d]['groups']:
        if i['name']:
            if i['name'] == 'shop':
                for s in i['categories']:
                    if 'shopify-app' in s['name']:
                        shopify_websites.append(relevent_domains[d]['domain'])

In [1062]:
# filtering out duplicates

target_websites = []

[target_websites.append(element) for element in shopify_websites if element not in target_websites]

target_websites

['naomi.se', 'sb-components.co.uk', 'steverude.com', 'iamrunbox.com']

In [1063]:
# returning campaigns that use shopify

kickstarter_shopify = []

for i in range(len(kickstarter_id)):
    for site in target_websites:
        if site in str(kickstarter_id[i]['website']).lower():
            kickstarter_shopify.append(kickstarter_id[i])

#### getting contact information

In [1064]:
leads = {}

for website in target_websites:
    domain = website
    url = 'https://api.hunter.io/v2/domain-search?domain='+ website +'&api_key='
    r = requests.get(url)
    leads[website] = r.json()

In [1065]:
len(leads)

4

In [1094]:
contact_data = {}

for website in target_websites:

    contact_data[website] = {}
        
    for i in range(len(leads[website]['data']['emails'])):

        contact_data[website]['contact_' + str(i)] = {}

        contact_data[website]['contact_' + str(i)]['email'] = leads[website]['data']['emails'][i]['value']
        contact_data[website]['contact_' + str(i)]['first_name'] = leads[website]['data']['emails'][i]['first_name']
        contact_data[website]['contact_' + str(i)]['last_name']  = leads[website]['data']['emails'][i]['last_name']
        contact_data[website]['contact_' + str(i)]['position'] = leads[website]['data']['emails'][i]['position']

        contact_data[website]['contact_' + str(i)]['department'] = leads[website]['data']['emails'][i]['department']
        contact_data[website]['contact_' + str(i)]['seniority'] = leads[website]['data']['emails'][i]['seniority']
        contact_data[website]['contact_' + str(i)]['linkedin'] = leads[website]['data']['emails'][i]['linkedin']
        contact_data[website]['contact_' + str(i)]['twitter'] = leads[website]['data']['emails'][i]['twitter']
        contact_data[website]['contact_' + str(i)]['phone_number'] = leads[website]['data']['emails'][i]['phone_number']


#### adding data to contacts

In [1095]:
# adding relevant data to contact data

for key, value in contact_data.items():
    for i in range(len(kickstarter_shopify)):
        if key in str(kickstarter_shopify[i]['website']).lower():
            for k, v in value.items():
                v['backer_count'] = kickstarter_shopify[i]['backers_count']
                v['project_name'] = kickstarter_shopify[i]['name']

#### filtering relevant contacts

In [1097]:
# these are some ideas to filter by, we would need to expand these lists

departments = ['sales', 'marketing', 'communication', 'executive', 'customer']
positions = ['founder', 'ceo', 'operations', 'marketing', 'customer', 'ecommerce', 'e-commerce', 'e commerce']

# would be better to check if there is a partial match

In [1098]:
for key, value in contact_data.items():
    for k, v in list(value.items()):
        if v['first_name'] == None:
            del(value[k])
        
        # elif v['backer_count'] < 20:
            # del(value[k])'''
        
        elif v['department'] in departments or v['position'] in positions:
            pass

    if len(value) == 1:
            pass
    
    else:
        try:
            del(value[k])
        except:
            pass

In [1099]:
# removing empty dicts

for entry in list(contact_data):
    if bool(contact_data[entry]) == False:
        del(contact_data[entry])

#### copy draft

''' Dear {{first_name}} {{last_name}},

I am reaching out to you from Gorgias becuase we noticed that you have successfully finished your kickstarter campaign {{project_name}}!

Gorgias is a helpdesk company which allows its clients to easily and effectively keep track of and respond to customer interactions. 
We offer ways to automate customer service and can greatly increase your ability to repond to customers.

Gorgias is the best way to make sure you keep your {{backer_count}} backers informed and up to date.

If you are interested in trying our product please reach out to us by replying to this email or book your demo at gorgias.com/demo.

Thank you for your time,

Zachary Highton '''

#### Segment

In [1102]:
import analytics

analytics.write_key = ''

In [1103]:
for key, value in contact_data.items():
    for k, v in value.items():
        analytics.track(v['email'], 'kickstarter-campaign-completed',
        {'first_name': v['first_name'],
        'last_name':v['last_name'],
        'project_name':v['project_name'],
        'backer_count':v['backer_count']})